In [1]:
import numpy as np
import pandas as pd

In [50]:
what = "경영권분쟁소송"

In [51]:
df = pd.read_csv("label_data/{}_2012_2022.csv".format(what), index_col=0).sort_values("공시일",axis=0).reset_index(drop=True)

df = df.loc[~df["보고서명"].str.endswith("우)")]
df = df.loc[~df["보고서명"].str.endswith("우B)")]
df = df.loc[~df["보고서명"].str.endswith("CJ4우(전환))")]
df = df.loc[~df["보고서명"].str.contains("ETN")]

df2 = df.reset_index(drop=True)

In [52]:
df2.to_csv("label_data/{}_전체시장_우선주제거.csv".format(what))

In [77]:
### 투자주의 데이터 확인

#df_daq = pd.read_csv("투자주의_코스닥_2012-2022.csv", index_col=0)
df = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\투자주의_코스닥_2012-01-01-2012-12-31.csv", index_col=0)
df["보고서명"].unique()
#df.loc[df["보고서명"].str.contains("ETN")]

array(['[투자주의]소수지점ㆍ소수계좌 거래집중 종목', '[투자주의]종가급변종목', '[투자주의]상한가잔량 상위종목',
       '[투자주의]단일계좌 거래량 상위종목', '[투자주의]15일간 예방조치 과다종목',
       '[투자주의]15일간 상승종목의 당일 소수계좌 매수관여 과다종목'], dtype=object)

In [71]:
df = pd.merge(df_daq,df_pi, left_on="현재공시코드",right_on="현재공시코드", how="outer")

df.loc[df.isna().sum(axis=1) >=1]["보고서명_x"].unique()

array(['[ETN 투자주의]소수지점ㆍ소수계좌 거래집중 종목', '[ETN 투자주의]특정계좌(군) 매매관여 과다종목'],
      dtype=object)

## 2012-2022 데이터를 합치기

In [162]:
full = pd.DataFrame()
for year in range(2012,2023):
    globals()["df"+str(year)] = pd.read_csv("투자주의_코스닥_{}-01-01-{}-12-31.csv".format(year,year), index_col=0)
    full = pd.concat([globals()["df"+str(year)], full])
full = full.sort_values("공시일").reset_index(drop=True)

In [168]:
full["기업코드"] = full["기업코드"].apply(lambda x: "A"+str(x).zfill(6))

In [170]:
full.to_csv("투자주의_코스닥_2012_2022.csv")

### 코스닥, 코스피 데이터 합치기

In [173]:
df1 = pd.read_csv("투자주의_코스닥_2012_2022.csv", index_col=0)
df2 = pd.read_csv("투자주의_코스피_2012-2022.csv", index_col=0)

In [176]:
df3 = pd.concat([df1,df2]).sort_values("공시일").reset_index(drop=True)

df3.to_csv("투자주의_2012_2022.csv")

## 데이터 전체 점검

In [79]:
import os

In [142]:
path = "C:\\JeHyeon_File\\데이터\\KIND데이터(2012_2022)"
file_name_list = os.listdir(path)
file_dict = {}
for name in file_name_list:
    file_dict[name[:-14]] = pd.read_csv("{}\\{}".format(path, name), index_col=0)
file_dict.keys()

dict_keys(['BW', 'CB', '관리종목', '불성실공시법인지정', '상호변경', '업종변경', '자기주식처분결정', '조회공시요구', '최대주주변경', '타법인주식및출자증권취득결정', '투자경고종목지정', '투자위험종목지정', '투자주의환기종목지정', '특수관계인에대한자금대여', '횡령배임사실확인', '횡령배임혐의발생', '횡령배임혐의발생_201'])

In [139]:
read_what = "투자위험종목지정"
df = file_dict[read_what].loc[~file_dict[read_what]["보고서명"].str.endswith("우)")].reset_index(drop=True)
df = df.loc[~df["보고서명"].str.endswith("우B)")].reset_index(drop=True)
#df.to_csv("{}\\{}.csv".format(path,name))
df

,회사명,기업공시코드,공시일,보고서명
0,팜젠사이언스,A004720,2012-02-15,투자위험종목 지정
1,써니전자,A004770,2012-04-20,투자위험종목 지정
2,써니전자,A004770,2013-01-15,투자위험종목 지정
3,브이티지엠피,A018290,2012-06-19,투자위험종목 지정
4,녹십자셀,A031390,2012-07-04,투자위험종목 지정
...,...,...,...,...
85,하이드로리튬,A101670,2022-09-30,투자위험종목 지정
86,베트남개발1,A096300,2022-10-06,투자위험종목 지정
87,어반리튬,A073570,2022-10-06,투자위험종목 지정
88,인디에프,A014990,2022-11-10,투자위험종목 지정


In [137]:
df.to_csv("투자위험종목지정_2012_2022.csv")

In [140]:
file_dict[read_what]["보고서명"].unique()

array(['투자위험종목 지정'], dtype=object)

### 금융 시계열 형태로 변환하기

* 투자주의 투자위험 투자주의환기종목 투자경고종목

In [14]:
what  = "경영권분쟁소송"
df = pd.read_csv("Use_data/{}_2012_2022.csv".format(what), index_col=0)

#df["기업공시코드"] = df["기업공시코드"].apply(lambda x: "A"+str(x).zfill(6))
# zfill(6)으로 변경
df
cross_df = pd.crosstab(df["공시일"], df["기업공시코드"])# values=df["양수도대금(원)"], aggfunc="sum", dropna=False).fillna(0)
cross_df.to_csv("{}_전체시장.csv".format(what))

In [28]:
for col in cross_df.columns:
    if "_x" in col or "_y" in col:
        print(col)

라벨수정

In [207]:
cross_df = { "횡령":cross_df}

In [208]:
cross_df

{'횡령': 기업공시코드      A000240  A000360  A000470  A000640  A000880  A001070  A001440  \
 공시일                                                                         
 2012-01-06        0        0        0        0        0        0        0   
 2012-01-10        0        0        0        0        0        0        0   
 2012-01-26        0        0        0        0        0        0        1   
 2012-02-03        0        0        0        0        1        0        0   
 2012-02-21        0        0        0        0        0        0        0   
 ...             ...      ...      ...      ...      ...      ...      ...   
 2022-11-29        0        0        0        0        0        0        0   
 2022-12-01        0        0        0        0        0        0        0   
 2022-12-06        0        0        0        0        0        0        0   
 2022-12-08        0        0        0        0        0        0        0   
 2022-12-21        0        0        0        0        0  

In [2]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster.kmeans import MyKmeans

In [5]:
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\Use_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\Use_data\\재무\\")

kind_data["BW금액_전체시장"]

,A000040,A000220,A000250,A000300,A000390,A000420,A000520,A000640,A000890,A001020,...,A304100,A317870,A321820,A332570,A348370,A900050,A900080,A900100,A900110,A900120
공시일,,,,,,,,,,,,,,,,,,,,,
2012-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
see = financial_data["총자본"]
see

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A950110,A950130,A950140,A950160,A950170,A950180,A950190,A950200,A950210,A950220
2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-28,2.360772e+09,1.263030e+08,2.657442e+09,-14553934.00,7.608496e+07,1.225921e+08,NaN,2.022708e+08,NaN,2.907332e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-31,2.360772e+09,1.263030e+08,2.657442e+09,-14553934.00,7.608496e+07,1.225921e+08,NaN,2.022708e+08,NaN,2.907332e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,2.955748e+10,3.652476e+08,2.412578e+10,47885407.29,7.687401e+08,1.009544e+09,2.471238e+09,1.112067e+09,NaN,1.935133e+09,...,42545499.0,661635739.0,90157735.0,33409905.0,24966475.0,241976301.0,104794634.0,34667007.0,429935856.0,140915532.0
2022-09-30,2.955748e+10,3.652476e+08,2.412578e+10,47885407.29,7.687401e+08,1.009544e+09,2.471238e+09,1.112067e+09,NaN,1.935133e+09,...,42545499.0,661635739.0,90157735.0,33409905.0,19096016.0,241976301.0,104794634.0,34667007.0,429935856.0,140915532.0
2022-10-31,3.050465e+10,3.720029e+08,2.544164e+10,43977765.59,7.650997e+08,6.180507e+08,2.474793e+09,1.148856e+09,NaN,1.938411e+09,...,44558144.0,710433597.0,103698916.0,104526359.0,19096016.0,241976301.0,119456669.0,39789178.0,638897118.0,134629385.0
2022-11-30,3.050465e+10,3.720029e+08,2.544164e+10,43977765.59,7.650997e+08,6.180507e+08,2.474793e+09,1.148856e+09,NaN,1.938411e+09,...,44558144.0,710433597.0,103698916.0,104526359.0,19096016.0,241976301.0,119456669.0,39789178.0,638897118.0,134629385.0


In [16]:
np.nanmean(see,axis=0)

C:\Users\user\AppData\Local\Temp\ipykernel_4604\2265508745.py:1: RuntimeWarning: Mean of empty slice
  np.nanmean(see,axis=0)


array([1.50253550e+10, 2.17614034e+08, 1.46719283e+10, ...,
       1.61794606e+07, 4.30700763e+08, 1.17037668e+08])

In [ ]:
see.loc[see.mam]

In [21]:
see

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A950110,A950130,A950140,A950160,A950170,A950180,A950190,A950200,A950210,A950220
2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-28,2.360772e+09,1.263030e+08,2.657442e+09,-14553934.00,7.608496e+07,1.225921e+08,NaN,2.022708e+08,NaN,2.907332e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-31,2.360772e+09,1.263030e+08,2.657442e+09,-14553934.00,7.608496e+07,1.225921e+08,NaN,2.022708e+08,NaN,2.907332e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,2.955748e+10,3.652476e+08,2.412578e+10,47885407.29,7.687401e+08,1.009544e+09,2.471238e+09,1.112067e+09,NaN,1.935133e+09,...,42545499.0,661635739.0,90157735.0,33409905.0,24966475.0,241976301.0,104794634.0,34667007.0,429935856.0,140915532.0
2022-09-30,2.955748e+10,3.652476e+08,2.412578e+10,47885407.29,7.687401e+08,1.009544e+09,2.471238e+09,1.112067e+09,NaN,1.935133e+09,...,42545499.0,661635739.0,90157735.0,33409905.0,19096016.0,241976301.0,104794634.0,34667007.0,429935856.0,140915532.0
2022-10-31,3.050465e+10,3.720029e+08,2.544164e+10,43977765.59,7.650997e+08,6.180507e+08,2.474793e+09,1.148856e+09,NaN,1.938411e+09,...,44558144.0,710433597.0,103698916.0,104526359.0,19096016.0,241976301.0,119456669.0,39789178.0,638897118.0,134629385.0
2022-11-30,3.050465e+10,3.720029e+08,2.544164e+10,43977765.59,7.650997e+08,6.180507e+08,2.474793e+09,1.148856e+09,NaN,1.938411e+09,...,44558144.0,710433597.0,103698916.0,104526359.0,19096016.0,241976301.0,119456669.0,39789178.0,638897118.0,134629385.0


### 유상증자합치기

* 주식담보

In [2]:
import numpy as np 
import pandas as pd
import os

In [4]:
lstdir = os.listdir("경영권분쟁소송/")
dict = {}
concat_df = pd.DataFrame()

for i,name in enumerate(lstdir):
    dict[i] = pd.read_csv("경영권분쟁소송/{}".format(name),index_col=0)
    concat_df = pd.concat([concat_df,dict[i]])

In [12]:
df = pd.read_csv("기업명_기업코드.csv", index_col=0)
a = pd.merge(concat_df, df, left_on="회사명",right_on="회사명").drop(["Dart코드","코드"], axis=1).sort_values("공시일").reset_index(drop=True)
a.to_csv("경영권분쟁소송_2012_2022.csv")

In [5]:
concat_df

,공시일,회사명,코드,보고서명,정정신고
0,2014-12-29,신일전자,20141229000587,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO
1,2014-12-22,참엔지니어링,20141222000626,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO
2,2014-12-22,참엔지니어링,20141222000616,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO
3,2014-12-22,참엔지니어링,20141222000520,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO
4,2014-12-15,신일전자,20141215000972,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO
...,...,...,...,...,...
368,2021-01-11,팍스넷,20210111000142,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO
369,2021-01-08,삼영이엔씨,20210108000423,소송 등의 제기ㆍ신청(경영권 분쟁 소송)(의결권행사금지가처분),NO
370,2021-01-07,제이스코홀딩스,20210107000415,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO
371,2021-01-07,제이스코홀딩스,20210107000316,소송 등의 제기ㆍ신청(경영권 분쟁 소송),NO


In [9]:
d0 = dict["주식양수도_코스닥_2012"]
d1 = dict["주식양수도_코스닥_2015"]
d2 = dict["주식양수도_코스닥_2018"]
d3 = dict["주식양수도_코스닥_2021"]


done = pd.concat([d0,d1,d2, d3])
done["기업공시코드"] = done["기업공시코드"].apply(lambda x: "A" + str(x).zfill(6))
done = done.sort_values("공시일").reset_index(drop=True)

done.to_csv("주식양수도_2012_2022.csv")

In [44]:
d0 = dict["주식담보_코스닥_2015"]
d1 = dict["주식담보_코스닥_2018"]
d2 = dict["주식담보_코스닥_2021"]

done = pd.concat([d0,d1,d2])
done["기업코드"] = done["기업코드"].apply(lambda x: "A" + str(x).zfill(6))
done = done.sort_values("공시일").reset_index(drop=True)
done.columns = ['회사명', '기업공시코드', '공시일', '차입금액총액(원)', '담보설정금액(원)', '정정공시코드', '현재공시코드',
       '차입+담보(원)']
#done.to_csv("주식담보양수도_2012_2022.csv")

In [13]:
d0 = dict["유상증자_코스닥_2012"]
d1 = dict["유상증자_코스닥_2014"]
d2 = dict["유상증자_코스닥_2017"]
d3 = dict["유상증자_코스닥_2020"]

concat_df = pd.concat([d0,d1,d2,d3])

concat_df["발행가(원)"] = concat_df["발행금액"]
concat_df["발행주식수"] = concat_df["발행주식수"].str.replace(",","").astype("int64")
concat_df["발행가(원)"] = concat_df["발행가(원)"].str.replace(",","").astype("int64")

concat_df["발행금액"] = concat_df["발행주식수"] * concat_df["발행가(원)"]

concat_df

,회사명,기업공시코드,공시일,발행방법,주식종류,발행주식수,발행금액,발행가(원)
0,팬스타엔터프라이즈,54300,2013-12-16,일반공모,보통주,5982000,3684912000,616
1,THE E&M,89230,2013-12-13,제3자배정,보통주,3570527,8283622640,2320
2,안국약품,1540,2013-12-05,주주배정 후 실권주 일반공모,보통주,939597,6473823330,6890
3,엘컴텍,37950,2013-11-21,제3자배정,보통주,56200000,28100000000,500
4,엘컴텍,37950,2013-11-21,제3자배정,보통주,170922537,85461268500,500
...,...,...,...,...,...,...,...,...
310,아이톡시,52770,2020-01-16,제3자배정,보통주,9441499,4720749500,500
311,아이윈플러스,123010,2020-01-10,제3자배정,보통주,583060,3200999400,5490
312,커머스마이너,223310,2020-01-09,일반공모,보통주,8196721,999999962,122
313,하이소닉,106080,2020-01-07,제3자배정,보통주,3999996,3999996000,1000


In [17]:
d4 = dict["유상증자_코스피_2012"]
d5 = dict["유상증자_코스피_2014"]
d6 = dict["유상증자_코스피_2017"]
d7 = dict["유상증자_코스피_2020"]

concat_kospi = pd.concat([d4,d5,d6,d7])
concat_kospi

,회사명,기업공시코드,공시일,발행가(원),발행방법,주식종류,발행주식수,발행금액
0,STX중공업,71970,2013-12-17,2500.0,유상증자(제3자배정),기명식 보통,70882000.0,1.772050e+11
1,현대두산인프라코어,42670,2013-12-17,10845.0,유상증자(DR),기명식 보통,38797280.0,4.207565e+11
2,STX조선해양,67250,2013-12-04,2500.0,유상증자(제3자배정),기명식 보통,15573046.0,3.893262e+10
3,DRB동일,4840,2013-11-28,500.0,유상증자(공모증자),기명식 보통,4269461.0,2.134730e+09
4,한화손해보험,370,2013-11-25,3740.0,유상증자(주주우선공모증자),기명식 보통,42000000.0,1.570800e+11
...,...,...,...,...,...,...,...,...
248,SK렌터카,68400,2020-01-17,11932.0,유상증자(제3자배정),기명식 보통,13618840.0,1.625000e+11
249,금호에이치티,214330,2020-01-13,5160.0,유상증자(제3자배정),기명식 보통,1937985.0,1.000000e+10
250,쎌마테라퓨틱스,15540,2020-01-10,4880.0,유상증자(제3자배정),기명식 보통,1484423.0,7.243984e+09
251,KH 필룩스,33180,2020-01-09,7100.0,유상증자(제3자배정),기명식 보통,1549296.0,1.100000e+10


In [19]:
done = pd.concat([concat_df, concat_kospi])
done["기업공시코드"] = done["기업공시코드"].apply(lambda x: "A" + str(x).zfill(6))
done = done.sort_values("공시일").reset_index(drop=True)

In [21]:
done.to_csv("유상증자_2012_2022.csv")

## 데이터를 Shift 시켜보자

In [13]:
import os
import pandas as pd
import numpy as np

In [82]:
path_list = os.listdir("label_data/")
dict = {}
for path in path_list:
    dict[str(path)] = pd.read_csv("label_data/{}".format(path))


In [16]:
df = pd.read_csv("관리종목_전체시장.csv", index_col=0)
df

,A000360,A000420,A000800,A001000,A001260,A001310,A001380,A001440,A001470,A001520,...,A299910,A306620,A318020,A323280,A900100,A900110,A900250,A900290,A950160,A950170
공시일,,,,,,,,,,,,,,,,,,,,,
2012-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-01-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-14,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [19]:
df.loc["2022-12-30"].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [23]:
df.index = pd.to_datetime(df.index)
save = df.copy().shift(-1, freq='M')
#df.loc["2022-12-30"]
save.to_csv("관리종목지정_shift.csv")

In [5]:
save

,A000240,A000360,A000470,A000640,A000880,A001070,A001440,A001470,A001520,A001740,...,A215600,A219750,A220630,A223310,A257370,A263920,A267850,A298690,A352940,A950160
공시일,,,,,,,,,,,,,,,,,,,,,
2011-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-12-31,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-01-31,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-11-30,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [75]:
dict["투자경고종목지정카운트_전체시장.csv"].index = pd.to_datetime(dict["투자경고종목지정카운트_전체시장.csv"]["공시일"])

save = dict["투자경고종목지정카운트_전체시장.csv"].copy().shift(-1, freq='M')
#pd.concat([dict["투자경고종목지정카운트_전체시장.csv"].shift(1), save]).drop_duplicates()

In [81]:
for key, df in dict.items():
    df.index = pd.to_datetime(df["공시일"])
    save = df.copy().shift(-1, freq='M').drop("공시일",axis=1)
    save.to_csv("{}_shift.csv".format(str(key)[:-9]))

### 주가 처리해보기

In [7]:
import numpy as np
import pandas as pd
from krx_fr.preprocess import tools, scaler

In [8]:
stock = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\수정주가.csv", index_col=0)
market = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\지수_종가.csv", index_col=0)

In [9]:
market.head()

,코스피,코스닥
2011-12-29,1825.74,500.18
2012-01-02,1826.37,506.79
2012-01-03,1875.41,513.83
2012-01-04,1866.22,516.30
2012-01-05,1863.74,521.96


In [10]:
stock.head()

,A000010,A000020,A000030,A000040,A000050,A000060,A000070,A000080,A000090,A000100,...,A950110,A950130,A950140,A950160,A950170,A950180,A950190,A950200,A950210,A950220
2012-01-02,NaN,4085.0,NaN,3359.0,7078.0,11450.0,78400.0,26300.0,NaN,18783.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-03,NaN,4130.0,NaN,3840.0,7108.0,11850.0,82500.0,26950.0,NaN,18710.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,NaN,4150.0,NaN,4414.0,7055.0,11800.0,82900.0,27250.0,NaN,18931.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-05,NaN,4270.0,NaN,4224.0,7093.0,11600.0,83400.0,27950.0,NaN,19299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-06,NaN,4240.0,NaN,4321.0,7093.0,11400.0,83700.0,27750.0,NaN,18857.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

In [12]:
concat_together = tools.dict_data_match_listed(stock.copy(), together_listed, "Y")

KeyError: 'A000010'